In [2]:
!wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py

--2023-04-26 21:20:48--  https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 0.0.0.0, ::
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|0.0.0.0|:443... failed: Connection refused.
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|::|:443... failed: Connection refused.


In [3]:
import torch
from unixcoder import UniXcoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UniXcoder("microsoft/unixcoder-base")
model.to(device)

UniXcoder(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(51416, 768, padding_idx=1)
      (position_embeddings): Embedding(1026, 768, padding_idx=1)
      (token_type_embeddings): Embedding(10, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [4]:
import pandas as pd
import numpy as np
import  re
from torch import per_channel_affine
from tqdm import tqdm
import time, random
import torch

import json
import cv2
import numpy as np
from tqdm import tqdm
import math
from math import log
from torch.nn.utils.rnn import pad_sequence
import sys
import time
import os
from collections import defaultdict, Counter
from multiprocessing import Pool
from functools import partial
from itertools import chain

获取数据文件（先找个小一点的jsonl文件放到服务器上，选择CSN里ruby语言的test文件比较小）

In [5]:
from transformers import AutoTokenizer, AutoModel

# Load pre-trained unixcoder tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('microsoft/unixcoder-base')
model = AutoModel.from_pretrained('microsoft/unixcoder-base')


In [6]:
#input:code snippet
import jsonlines

test_list=[]
test_file="/home/huangyx/emscore/emscore/dataset/ruby_test_0.jsonl"
with open(test_file,mode='r') as f:
    for item in jsonlines.Reader(f):
        test_list.append(item)
print(len(test_list))
    

2279


In [7]:
#gain sample
import json

sample_list=random.sample(test_list,10)
print(sample_list[0]['code'])
print("----------------------------------------")
print(sample_list[0]['docstring'])


def build_sale_data(sale_item)
        "".b.tap() do |data|
          price_units = (sale_item.price * 100).to_i # !FIXME
          price_bytes = "".b
          4.times{ |shift| price_bytes.insert 0, ((price_units >> shift*8) & 0xff).chr }
          data << price_bytes
          qty_units = ((sale_item.qty || 1) * 1000).to_i # !FIXME
          qty_bytes = "".b
          4.times{ |shift| qty_bytes.insert 0, ((qty_units >> shift*8) & 0xff).chr }
          data << qty_bytes
          data << "\x00".b #number len FIXME
          data << "\xAA\xAA\xAA\xAA\xAA\xAA".b #number FIXME
          text = sale_item.text1.truncate(20)
          data << text.length.chr
          data << text.ljust(20, " ").b
          data << (sale_item.tax_group || 2).chr
        end
      end
----------------------------------------
def build_sale_data(price, text1 = "", text2 = nil, tax_group = 2, qty = 1, percent = nil, neto = nil, number = nil)


In [10]:
#得到code和candidate的embedding向量,output_code/ouput_candidate就包含了每一个token/word向量
#code_embeddings  candidate_embeddings就是分别对应整个向量表示，这里取的是mean操作获得



# Encode input code and obtain word embeddings
input_code = sample_list[0]['code']
encoded_code = tokenizer.encode_plus(input_code, return_tensors='pt')
output_code = model(**encoded_code)[0]
code_embeddings = output_code.mean(dim=1)

# Encode input candidate and obtain word embeddings
input_candidate = sample_list[0]['docstring']
encoded_candidate = tokenizer.encode_plus(input_candidate, return_tensors='pt')
output_candidate = model(**encoded_candidate)[0]
candidate_embeddings = output_candidate.mean(dim=1)


In [8]:
print(test_list[1]['code'] )
print("-------------------")
print(test_list[1]['docstring'])

def print_summary(status)
      status_string = status.to_s.humanize.upcase
      if status == :success
        heading("Result: ", status_string, :green)
        level = :info
      elsif status == :timed_out
        heading("Result: ", status_string, :yellow)
        level = :fatal
      else
        heading("Result: ", status_string, :red)
        level = :fatal
      end

      if (actions_sentence = summary.actions_sentence.presence)
        public_send(level, actions_sentence)
        blank_line(level)
      end

      summary.paragraphs.each do |para|
        msg_lines = para.split("\n")
        msg_lines.each { |line| public_send(level, line) }
        blank_line(level) unless para == summary.paragraphs.last
      end
    end
-------------------
Outputs the deferred summary information saved via @logger.summary.add_action and @logger.summary.add_paragraph


In [ ]:
print(output_code.shape)
print(output_candidate.shape)

In [9]:
#这里的a参数指的是每一个candidate文本句子

def process(a, tokenizer=None):
    if tokenizer is not None:
        a1 = tokenizer.encode(a, add_special_tokens=True, return_tensors="pt").tolist()[0]
    return set(a1)

In [10]:
#算词频，一共N个句子，统计单词在这些句子里出现几次，出现在一个句子里算一次，输入一个corpus里所有句子，输出所有单词对应的词频

def get_idf_dict(arr, tokenizer, nthreads=4):
    """
    Returns mapping from word piece index to its inverse document frequency.
    Args:
        - :param: `arr` (list of str) : sentences to process.
        - :param: `tokenizer` : a BERT tokenizer corresponds to `model`.
        - :param: `nthreads` (int) : number of CPU threads to use
    """
    idf_count = Counter()
    num_docs = len(arr)

    process_partial = partial(process, tokenizer=tokenizer)

    with Pool(nthreads) as p:
        idf_count.update(chain.from_iterable(p.map(process_partial, arr)))

    idf_dict = defaultdict(lambda: log((num_docs + 1) / (1)))
    idf_dict.update({idx: log((num_docs + 1) / (c + 1)) for (idx, c) in idf_count.items()})
    return idf_dict

In [11]:
candidate_list=[]
for item in test_list:
    candidate_list.append(item['docstring'])



In [ ]:
#计算得到每个token对应的idf结果

idf_dict=get_idf_dict(candidate_list,tokenizer)
print(idf_dict)

In [ ]:
code_list=[]
for item in test_list:
    code_list.append(item['code'])
code_idf_dict=get_idf_dict(code_list,tokenizer)
print(code_idf_dict)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/unixcoder-base')
model = AutoModel.from_pretrained('microsoft/unixcoder-base')

In [15]:
def encode_text(vid_caps, model, tokenizer, idf_dict, device):
    with torch.no_grad():
        text_input = tokenizer.encode_plus(vid_caps, return_tensors='pt',return_attention_mask=True)

        text_features = model(**text_input)[0]
    #mask= text_input['attention_mask']

    text_features /= text_features.norm(dim=-1, keepdim=True)

    # For special tokens, use [SOS] and [EOS]
    txt_len = text_input['input_ids'].argmax(dim=-1)
    mask = torch.zeros_like(text_input['input_ids'])
    for i in range(len(mask)):
        mask[i][0:txt_len[i]+1] = 1

    idf_weights = torch.tensor([[idf_dict[int(i)] for i in a] for a in text_input['input_ids'].cpu()])

    return text_features, mask, idf_weights


In [23]:
vid_caps='Outputs the deferred summary information saved via @logger.summary.add_action and @logger.summary.add_paragraph'

#vid_caps='A person is frying a pan of eggs that are colored pink and green.'
device="cuda:0"
text_features,mask,idf_weights=encode_text(vid_caps,model,tokenizer,idf_dict,device)
print("文本处理之后的一些信息：")
print(type(text_features))
print(text_features)
print(text_features.shape)

print("关于mask的一些信息")
print(type(mask))
print(mask)
print(mask.shape)

print("关于idf权重的一些信息：")
print(type(idf_weights))
print(idf_weights)
print(idf_weights.shape)

文本处理之后的一些信息：
<class 'torch.Tensor'>
tensor([[[-0.0204, -0.0282, -0.0100,  ..., -0.0776, -0.0652, -0.0834],
         [-0.0204, -0.0282, -0.0100,  ..., -0.0776, -0.0652, -0.0834],
         [-0.0204, -0.0282, -0.0100,  ..., -0.0776, -0.0652, -0.0834],
         ...,
         [-0.0229,  0.0105,  0.0403,  ..., -0.0697, -0.0601, -0.0484],
         [ 0.0118,  0.0317,  0.0442,  ..., -0.0386, -0.0558, -0.0312],
         [-0.0548, -0.0081,  0.0229,  ..., -0.0307, -0.0644, -0.0186]]])
torch.Size([1, 26, 768])
关于mask的一些信息
<class 'torch.Tensor'>
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0]])
torch.Size([1, 26])
关于idf权重的一些信息：
<class 'torch.Tensor'>
tensor([[0.0000, 5.9402, 0.5549, 7.0388, 6.3456, 3.7246, 5.3340, 4.6409, 1.1307,
         5.4293, 0.5534, 7.0388, 0.5534, 4.3646, 1.1569, 5.6525, 1.3969, 1.1307,
         5.4293, 0.5534, 7.0388, 0.5534, 4.3646, 1.1569, 6.6333, 0.0000]])
torch.Size([1, 26])


对文本处理的部分处理完了，接下来应该把原来对video进行处理的部分改成对code进行处理，但本质上对code处理的过程应该跟text很类似，只是得到code中每个token的向量后，如何将其组合得到不同粒度的向量表示

In [21]:
#第一步应该类似text的操作，先获得code中每个token的embedding（token_level）以及整个code snippet的embedding表示（function_level）
def encode_code(code, model, tokenizer, code_idf_dict, device):
    with torch.no_grad():
        code_input = tokenizer.encode_plus(code, return_tensors='pt',return_attention_mask=True)

        code_features = model(**code_input)[0]


    code_features /= code_features.norm(dim=-1, keepdim=True)

    # For special tokens, use [SOS] and [EOS]
    code_len = code_input['input_ids'].argmax(dim=-1)
    code_mask = torch.zeros_like(code_input['input_ids'])
    for i in range(len(code_mask)):
        code_mask[i][0:code_len[i]+1] = 1

    code_idf_weights = torch.tensor([[code_idf_dict[int(i)] for i in a] for a in code_input['input_ids'].cpu()])

    return code_features, code_mask, code_idf_weights


In [22]:
code=test_list[1]['code']

device="cuda:0"
code_features,code_mask,code_idf_weights=encode_code(code,model,tokenizer,code_idf_dict,device)
print("代码处理之后的一些信息：")
print(type(code_features))
print(code_features)
print(code_features.shape)

print("关于mask的一些信息")
print(type(code_mask))
print(code_mask)
print(code_mask.shape)

print("关于idf权重的一些信息：")
print(type(code_idf_weights))
print(code_idf_weights)
print(code_idf_weights.shape)

代码处理之后的一些信息：
<class 'torch.Tensor'>
tensor([[[-0.0846, -0.0142,  0.0101,  ..., -0.0234,  0.0021, -0.0564],
         [-0.0846, -0.0142,  0.0101,  ..., -0.0234,  0.0021, -0.0564],
         [-0.0846, -0.0142,  0.0101,  ..., -0.0234,  0.0021, -0.0564],
         ...,
         [-0.0325,  0.0659,  0.0066,  ..., -0.0075,  0.0174, -0.0476],
         [-0.0238,  0.0480, -0.0140,  ...,  0.0212,  0.0016, -0.0897],
         [-0.0091, -0.0623,  0.0475,  ..., -0.0169,  0.0185,  0.0290]]])
torch.Size([1, 215, 768])
关于mask的一些信息
<class 'torch.Tensor'>
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
hyps=[]
hyps.append(vid_caps)

verbose=True
batch_size=64
return_matched_idx=False


refs_preds_local = []
refs_pred_matched_idxs = []
refs_preds_global = []

vid_preds_local = []
vid_pred_matched_idxs = []
vid_preds_global = []

"""process text"""
def dedup_and_sort(l):
    return sorted(list(set(l)), key=lambda x: len(x.split(" ")), reverse=True)

sentences = dedup_and_sort(hyps)
#sentences = dedup_and_sort(hyps)


embs = []
iter_range = range(0, len(sentences), batch_size)
if verbose:
    print("computing text embedding.")
    iter_range = tqdm(iter_range)
text_local_stats_dict = dict()
text_global_stats_dict = dict()
for batch_start in iter_range:
    sen_batch = sentences[batch_start: batch_start + batch_size]
    print(sen_batch)
    print(type(sen_batch))
    #实际处理文本内容在这里调用encode_text
    #此处将encode_text函数替换成处理代码任务中candidate的函数
    embs, masks, text_idfs = encode_text(sen_batch[0], model, tokenizer, idf_dict, device=device)
    embs = embs.cpu()
    masks = masks.cpu()

    #i枚举sentence里的个数，一个reference+一个candidate就是从0到1

    for i, sen in enumerate(sen_batch):
        sequence_len = masks[i].sum().item()
        
        print("sequence_len的值是：")
        print(sequence_len)

        # For special tokens, use [SOS] and [EOS]
        local_emb = embs[i, 0:sequence_len]
        global_emb = embs[i, sequence_len-1]
        idf = text_idfs[i, 0:sequence_len]

        print("此处获得local embedding向量：")
        print(local_emb)
        print(local_emb.shape)
        print(" 此处获得global embedding向量：")
        print(global_emb)
        print(global_emb.shape)
        print("获得的idf值：")
        print(idf)
        print(idf.shape)

        # For special tokens, don't use any
        # local_emb = embs[i, 1:sequence_len+1]
        # global_emb = embs[i, sequence_len+1]
        # idf = text_idfs[i, 1:sequence_len+1]

        # For special tokens, only use [EOS] 
        # local_emb = embs[i, 1:sequence_len+1]
        # global_emb = embs[i, sequence_len]
        # idf = text_idfs[i, 1:sequence_len+1]

        text_local_stats_dict[sen] = (local_emb, idf)
        text_global_stats_dict[sen] = global_emb

        

100%|██████████| 1/1 [00:00<00:00, 26.41it/s]

computing text embedding.
['Outputs the deferred summary information saved via @logger.summary.add_action and @logger.summary.add_paragraph']
<class 'list'>
sequence_len的值是：
25
此处获得local embedding向量：
tensor([[-0.0204, -0.0282, -0.0100,  ..., -0.0776, -0.0652, -0.0834],
        [-0.0204, -0.0282, -0.0100,  ..., -0.0776, -0.0652, -0.0834],
        [-0.0204, -0.0282, -0.0100,  ..., -0.0776, -0.0652, -0.0834],
        ...,
        [-0.0251,  0.0308,  0.0083,  ..., -0.0389, -0.0996, -0.0329],
        [-0.0229,  0.0105,  0.0403,  ..., -0.0697, -0.0601, -0.0484],
        [ 0.0118,  0.0317,  0.0442,  ..., -0.0386, -0.0558, -0.0312]])
torch.Size([25, 768])
 此处获得global embedding向量：
tensor([ 1.1834e-02,  3.1693e-02,  4.4249e-02, -2.4906e-02,  4.6708e-02,
        -3.3223e-02,  3.4815e-02,  4.3800e-03,  6.3374e-02,  2.3459e-02,
        -2.7429e-03, -6.7016e-02, -4.9030e-02,  5.8587e-02,  2.5201e-02,
         3.4271e-02, -6.8927e-02,  2.3633e-02,  3.0989e-02, -2.1847e-02,
         3.3886e-02,  7.595

In [30]:
ref_code=[]
ref_code.append(code)

In [31]:


"""process code"""
code_sent = dedup_and_sort(ref_code)
#sentences = dedup_and_sort(hyps)


embs = []
iter_range = range(0, len(code_sent), batch_size)
if verbose:
    print("computing code embedding.")
    iter_range = tqdm(iter_range)
code_local_stats_dict = dict()
code_global_stats_dict = dict()
for batch_start in iter_range:
    code_sen_batch = code_sent[batch_start: batch_start + batch_size]

    #实际处理文本内容在这里调用encode_code
    #此处将encode_code函数替换成处理代码任务中candidate的函数
    code_embs, code_masks, code_idfs = encode_code(code_sen_batch[0], model, tokenizer, code_idf_dict, device=device)
    code_embs = code_embs.cpu()
    code_masks = code_masks.cpu()

    #i枚举sentence里的个数，一个reference+一个candidate就是从0到1

    for i, sen in enumerate(code_sen_batch):
        sequence_len = code_masks[i].sum().item()
        
        print("sequence_len的值是：")
        print(sequence_len)

        # For special tokens, use [SOS] and [EOS]
        code_local_emb = code_embs[i, 0:sequence_len]
        code_global_emb = code_embs[i, sequence_len-1]
        code_idf = code_idfs[i, 0:sequence_len]

        print("此处获得local embedding向量：")
        print(code_local_emb)
        print(code_local_emb.shape)
        print(" 此处获得global embedding向量：")
        print(code_global_emb)
        print(code_global_emb.shape)
        print("获得的idf值：")
        print(code_idf)
        print(code_idf.shape)

        # For special tokens, don't use any
        # local_emb = embs[i, 1:sequence_len+1]
        # global_emb = embs[i, sequence_len+1]
        # idf = text_idfs[i, 1:sequence_len+1]

        # For special tokens, only use [EOS] 
        # local_emb = embs[i, 1:sequence_len+1]
        # global_emb = embs[i, sequence_len]
        # idf = text_idfs[i, 1:sequence_len+1]

        text_local_stats_dict[sen] = (local_emb, idf)
        text_global_stats_dict[sen] = global_emb

        

100%|██████████| 1/1 [00:00<00:00, 12.06it/s]

computing code embedding.
sequence_len的值是：
24
此处获得local embedding向量：
tensor([[-0.0846, -0.0142,  0.0101,  ..., -0.0234,  0.0021, -0.0564],
        [-0.0846, -0.0142,  0.0101,  ..., -0.0234,  0.0021, -0.0564],
        [-0.0846, -0.0142,  0.0101,  ..., -0.0234,  0.0021, -0.0564],
        ...,
        [-0.0210,  0.0374, -0.0201,  ..., -0.0516,  0.0178, -0.0334],
        [-0.0248,  0.0724, -0.0183,  ..., -0.0430,  0.0414, -0.0561],
        [-0.0397,  0.0296, -0.0115,  ..., -0.0814,  0.0539, -0.0125]])
torch.Size([24, 768])
 此处获得global embedding向量：
tensor([-3.9688e-02,  2.9626e-02, -1.1544e-02,  1.4652e-02,  5.2433e-02,
         8.3955e-02, -2.3682e-02, -3.1725e-02, -2.5731e-02, -3.5622e-03,
        -2.6908e-02, -3.0190e-02, -1.0007e-02, -3.6842e-02,  3.6083e-03,
         1.0192e-02,  6.4037e-02,  2.9288e-02,  3.7607e-02,  4.2263e-02,
        -1.0714e-02,  5.0626e-02,  7.7406e-03, -6.1101e-02,  7.4759e-02,
         2.9475e-02,  4.8208e-02,  1.1013e-02,  2.3555e-02,  7.5120e-03,
         5.5

In [ ]:
def pad_local_batch_stats(sen_batch, stats_dict, device):
    stats = [stats_dict[s] for s in sen_batch]
    emb, idf = zip(*stats)
    emb = [e.to(device) for e in emb]
    lens = [e.size(0) for e in emb]
    emb_pad = pad_sequence(emb, batch_first=True, padding_value=0.0)
    idf_pad = pad_sequence(idf, batch_first=True)

    def length_to_mask(lens):
        lens = torch.tensor(lens, dtype=torch.long)
        max_len = max(lens)
        base = torch.arange(max_len, dtype=torch.long).expand(len(lens), max_len)
        return base < lens.unsqueeze(1)

    pad_mask = length_to_mask(lens).to(device)
    return emb_pad, pad_mask, idf_pad

In [ ]:
def pad_vid_local_batch_stats(sen_batch, stats_dict, device):
    stats = [stats_dict[s] for s in sen_batch]
    emb = stats
    emb = [e.to(device) for e in emb]
    lens = [e.size(0) for e in emb]
    emb_pad = pad_sequence(emb, batch_first=True, padding_value=0.0)

    def length_to_mask(lens):
        lens = torch.tensor(lens, dtype=torch.long)
        max_len = max(lens)
        base = torch.arange(max_len, dtype=torch.long).expand(len(lens), max_len)
        return base < lens.unsqueeze(1)

    pad_mask = length_to_mask(lens).to(device)
    return emb_pad, pad_mask
    

In [ ]:
def pad_global_batch_stats(sen_batch, stats_dict, device):
    stats = [stats_dict[s] for s in sen_batch]
    emb = stats
    emb = [e.to(device) for e in emb]
    emb_pad = pad_sequence(emb, batch_first=True, padding_value=0.0)
    return emb_pad

这部分是用video作为ground truth计算匹配的分数，对比着这部分进行修改

In [ ]:
def vid_greedy_cos(ref_embedding, ref_masks, hyp_embedding, hyp_masks, hyp_idf, return_matched_idx):
    """
    Compute greedy matching based on cosine similarity.

    Args:
        - :param: `ref_embedding` (torch.Tensor):
                   embeddings of reference sentences, BxKxd,
                   B: batch size, K: longest length, d: bert dimenison.
        - :param: `ref_masks` (torch.LongTensor): BxKxK, BERT attention mask for
                   reference sentences.
        - :param: `hyp_embedding` (torch.Tensor):
                   embeddings of candidate sentences, BxKxd,
                   B: batch size, K: longest length, d: bert dimenison
        - :param: `hyp_masks` (torch.LongTensor): BxKxK, BERT attention mask for
                   candidate sentences.
    """
    # ref_embedding and hyp_embedding are aleady L2-normalized.

    batch_size = ref_embedding.size(0)
    sim = torch.bmm(hyp_embedding, ref_embedding.transpose(1, 2))
    masks = torch.bmm(hyp_masks.unsqueeze(2).float(), ref_masks.unsqueeze(1).float())
    masks = masks.expand(batch_size, -1, -1).contiguous().view_as(sim)
    masks = masks.float().to(sim.device)
    sim = sim * masks

    word_precision, matched_indices = sim.max(dim=2)
    word_recall = sim.max(dim=1)[0]

    hyp_idf.div_(hyp_idf.sum(dim=1, keepdim=True))
    precision_scale = hyp_idf.to(word_precision.device)
    P = (word_precision * precision_scale).sum(dim=1)
    R = word_recall.sum(dim=1)/ref_masks.sum(dim=1)
    F = 2 * P * R / (P + R)
    
    if return_matched_idx:
        return P, R, F, matched_indices
    else:
        return P, R, F, torch.zeros_like(P)



In [ ]:
 """ if video used as ground truth """
if vids:
    if verbose:
        print("computing greedy matching, video as ground truth.")
    iter_range = range(0, len(ori_cands), batch_size)    
    with torch.no_grad():
        for batch_start in iter_range: 
            batch_ori_hyp = ori_cands[batch_start: batch_start + batch_size]
            ori_hyp_stats_local = pad_local_batch_stats(batch_ori_hyp, text_local_stats_dict, device)
            ori_hyp_stats_global = pad_global_batch_stats(batch_ori_hyp, text_global_stats_dict, device)

            batch_ori_vids = ori_vids[batch_start: batch_start + batch_size]
            ori_vids_stats_local = pad_vid_local_batch_stats(batch_ori_vids, vid_local_stats_dict, device)
            ori_vids_stats_global = pad_global_batch_stats(batch_ori_vids, vid_global_stats_dict, device)

            #在此处计算分数
            P, R, F1, matched_indices = vid_greedy_cos(*ori_vids_stats_local, *ori_hyp_stats_local, return_matched_idx)
            vid_preds_local.append(torch.stack((P, R, F1), dim=-1).cpu())
            vid_pred_matched_idxs.append(matched_indices)

            vid_s_cogr = torch.bmm(ori_hyp_stats_global.unsqueeze(1), ori_vids_stats_global.unsqueeze(1).transpose(1, 2)).squeeze()
            vid_preds_global.append(vid_s_cogr)  

